<a href="https://colab.research.google.com/github/jameschartouni/arabic_translation/blob/xlm/XLM_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [66]:
#installation step
!pip install transformers
!pip install sentencepiece
!pip install bpemb
#creating the folders 
!mkdir data/
!mkdir data/AD_NMT-master
!mkdir data/train/
!mkdir data/test/
!mkdir data/val/
!mkdir data/vocab
!mkdir data/model

#fetching the pkl files
!wget --no-check-certificate 'https://docs.google.com/uc?export=download&id=1V9crCmqvgQcv0Sx2MCNWB9AET2j6M6FW' -O data/AD_NMT-master/english-Arabic-both.pkl
!wget --no-check-certificate 'https://docs.google.com/uc?export=download&id=1V8_tp8ZlWUYaX7QQL46t0uSRNrVehSf1' -O data/AD_NMT-master/english-Arabic-test.pkl
!wget --no-check-certificate 'https://docs.google.com/uc?export=download&id=1V7X0qtuDIyjTHY0wh-ZNoVwsiF4lId2e' -O data/AD_NMT-master/english-Arabic-train.pkl
!wget --no-check-certificate 'https://docs.google.com/uc?export=download&id=1UzL4cOWTMCee83KBUh2QO_H62AFVpDQV' -O data/AD_NMT-master/LAV-MSA-2-both.pkl
!wget --no-check-certificate 'https://docs.google.com/uc?export=download&id=1UpfCbkxhztof7dvNjeAs1bHjD4SER6h3' -O data/AD_NMT-master/LAV-MSA-2-test.pkl
!wget --no-check-certificate 'https://docs.google.com/uc?export=download&id=1UlAZGtYsSfXzK7hrC_PbxQFqTSXD0DMw' -O data/AD_NMT-master/LAV-MSA-2-train.pkl
!wget --no-check-certificate 'https://docs.google.com/uc?export=download&id=1UjDX7cCG2S23SPfSHxSPdVayMTxB5Y16' -O data/AD_NMT-master/Magribi_MSA-both.pkl
!wget --no-check-certificate 'https://docs.google.com/uc?export=download&id=1UaVWIqRXo0rxuxDF4KArA4bEK1TaLX3l' -O data/AD_NMT-master/Magribi_MSA-test.pkl
!wget --no-check-certificate 'https://docs.google.com/uc?export=download&id=1UYvlhdYAdfa4riP_4hn3-IEVd1ZUXVTQ' -O data/AD_NMT-master/Magribi_MSA-train.pkl

mkdir: cannot create directory ‘data/’: File exists
mkdir: cannot create directory ‘data/AD_NMT-master’: File exists
mkdir: cannot create directory ‘data/train/’: File exists
mkdir: cannot create directory ‘data/test/’: File exists
mkdir: cannot create directory ‘data/val/’: File exists
mkdir: cannot create directory ‘data/vocab’: File exists
mkdir: cannot create directory ‘data/model’: File exists
--2020-05-22 14:38:30--  https://docs.google.com/uc?export=download&id=1V9crCmqvgQcv0Sx2MCNWB9AET2j6M6FW
Resolving docs.google.com (docs.google.com)... 108.177.126.139, 108.177.126.102, 108.177.126.101, ...
Connecting to docs.google.com (docs.google.com)|108.177.126.139|:443... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://doc-10-2s-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/hjkcpit4qlus17bab5ompdgk214mu1t2/1590158250000/16970776037313924126/*/1V9crCmqvgQcv0Sx2MCNWB9AET2j6M6FW?e=download [following]
--2020-05-22 14:3

In [0]:
#James Chartouni
#Joey Park
#Raef Khan

import torch
from torch.optim import SGD
from torch import nn, optim
from torchtext import data
import pandas as pd
import numpy as np
import pickle
import os, io, glob
import functools
from sklearn.model_selection import train_test_split

from bpemb import BPEmb

import transformers
from transformers import XLMTokenizer, XLMModel
from transformers import T5Tokenizer, T5Model, T5Config
import torchtext
from torchtext.data import Field, BucketIterator
from torchtext.datasets import TranslationDataset


import sentencepiece as spm

import time

In [0]:
file_path = 'data/AD_NMT-master/'

with open(file_path + "english-Arabic-train.pkl", 'rb') as handle:
    data_English_MSA_trainval = pickle.load(handle)

with open(file_path + "english-Arabic-test.pkl", 'rb') as handle:
    data_English_MSA_test = pickle.load(handle)

with open(file_path + "english-Arabic-both.pkl", 'rb') as handle:
    data_English_MSA_both = pickle.load(handle) 




with open(file_path + "LAV-MSA-2-train.pkl", 'rb') as handle:
    data_LAV_MSA_trainval = pickle.load(handle) 

with open(file_path + "LAV-MSA-2-test.pkl", 'rb') as handle:
    data_LAV_MSA_test = pickle.load(handle) 

with open(file_path + "LAV-MSA-2-both.pkl", 'rb') as handle:
    data_LAV_MSA_both = pickle.load(handle) 




with open(file_path + "Magribi_MSA-train.pkl", 'rb') as handle:
    data_Magribi_MSA_trainval = pickle.load(handle) 
    
with open(file_path + "Magribi_MSA-test.pkl", 'rb') as handle:
    data_Magribi_MSA_test = pickle.load(handle) 

with open(file_path + "Magribi_MSA-both.pkl", 'rb') as handle:
    data_Magribi_MSA_both = pickle.load(handle) 
    

In [69]:
print(data_English_MSA_both[0:5])
print(data_LAV_MSA_both[0:5])
print(len(data_English_MSA_trainval))
print(len(data_English_MSA_both))

[['Tom was also there', 'كان توم هنا ايضا'], ['That old woman lives by herself', 'تلك المراة العجوز تسكن بمفردها'], ['He went abroad for the purpose of studying English', 'سافر خارج البلد ليتعلم الانجليزية'], ['There is a fork missing', 'هناك شوكة ناقصة'], ["I don't know this game", 'لا اعرف هذه اللعبة']]
[['لا انا بعرف وحدة راحت ع فرنسا و معا شنتا حطت فيها الفرش', 'لا اعرف واحدة ذهبت الى فرنسا و لها غرفة و ضعت فيها الافرشة'], ['روح بوشك و فتول عاليسار', 'اذهب تقدم و استدر يسارا'], ['لا لا لازم انه يكون عندك موضوع ما في اشي', ' لا لا يجب ان يكون لديك موضوع هذا ضروري'], ['اوعي تبعدي من هون بلاش تضيعي ', 'لا تبتعد عن هنا حتى لا تفقد الطريق '], ['قصدي صراحة يما انا كمان كرهته من يوم ما عملتيه زي ما بتعمله خالتي كرهته و صرت ما باطيقه بالمرة', 'اقصد صراحة يا امي انا ايضا كرهته من يوم حضرته مثلما تحضره خالتي كرهته و اصبحت لا اطيقه ابدا']]
9000
10001


##Prepare Datasets

SPM Tokenizer

In [70]:
ls data/AD_NMT-master/

english-Arabic-both.pkl   LAV-MSA-2-both.pkl   Magribi_MSA-both.pkl
english-Arabic-test.pkl   LAV-MSA-2-test.pkl   Magribi_MSA-test.pkl
english-Arabic-train.pkl  LAV-MSA-2-train.pkl  Magribi_MSA-train.pkl


In [0]:
#splits the train dataset into train and validation sets, define test set as datafile
eng_msa_train, eng_msa_val = train_test_split(data_English_MSA_trainval, test_size=.2, random_state=22)
eng_msa_test = data_English_MSA_test

lav_msa_train, lav_msa_val = train_test_split(data_LAV_MSA_trainval, test_size=.2, random_state=22)
lav_msa_test = data_LAV_MSA_test

mag_msa_train, mag_msa_val = train_test_split(data_Magribi_MSA_trainval, test_size=.2, random_state=22)
mag_msa_test = data_Magribi_MSA_test

In [72]:
print(len(eng_msa_train))
print(len(eng_msa_val))

print(len(lav_msa_train))
print(len(lav_msa_val))

print(len(mag_msa_train))
print(len(mag_msa_val))

7200
1800
11044
2761
14188
3548


In [0]:
file_path = 'data/'

def pytorch_format(ds, src='en', trg='msa', datatype=''):
    src_formatted = datatype + '_' + src + '_' + trg + '.' + src
    trg_formatted = datatype + '_' + src + '_' + trg + '.' + trg
    
    with open(file_path + datatype + "/" + src_formatted, 'wt') as srctxt, open(file_path + datatype + "/" + trg_formatted, 'wt') as trgtxt:
        for i, arr in enumerate(ds):
            #srctxt.write(datatype.upper() + '_' + str(i).zfill( len(str(len(ds))) - len(str(i))) + '\\01\\' + arr[0] + '\n')
            #trgtxt.write(datatype.upper() + '_' + str(i).zfill( len(str(len(ds))) - len(str(i))) + '\\01\\' + arr[1] + '\n')
            srctxt.write(arr[0] + '\n')
            trgtxt.write(arr[1] + '\n')

In [0]:
#splits each language pair file into datasets of single language, to be merged again by the pytorch dataset class later

pytorch_format(eng_msa_train, 'eng', 'msa', 'train')
pytorch_format(eng_msa_val, 'eng', 'msa', 'val')
pytorch_format(eng_msa_test, 'eng', 'msa', 'test')

pytorch_format(lav_msa_train, 'lav', 'msa', 'train')
pytorch_format(lav_msa_val, 'lav', 'msa', 'val')
pytorch_format(lav_msa_test, 'lav', 'msa', 'test')

pytorch_format(mag_msa_train, 'mag', 'msa', 'train')
pytorch_format(mag_msa_val, 'mag', 'msa', 'val')
pytorch_format(mag_msa_test, 'mag', 'msa', 'test')

In [0]:
"""
Testing:
Create a text file with all the vocab available from all sources for each language for SentencePiece to create a library 

TODO: implement bpemb model for MSA vocab/train SPM on larger datasets for dialects
"""

en_vocab = open("data/vocab/eng_vocab.txt", "wt")
msa_vocab = open("data/vocab/msa_vocab.txt", "wt")
lav_vocab = open("data/vocab/lav_vocab.txt", "wt")
mag_vocab = open("data/vocab/mag_vocab.txt", "wt")

MSA_text = ""
EN_text = ""

def create_vocab(file='', src='en_vocab', tgt='msa_vocab'):
  for line in file:
        src_sent = line[0]
        src_words = src_sent.split(" ")
        for count, word in enumerate(src_words):
            src.write(word)
        src.write("\n")
        
        tgt_sent = line[1]
        tgt_words = tgt_sent.split(" ")
        for count, word in enumerate(tgt_words):
            tgt.write(word)
        tgt.write("\n")

create_vocab(data_English_MSA_both, en_vocab, msa_vocab)
create_vocab(data_LAV_MSA_both, lav_vocab, msa_vocab)
create_vocab(data_Magribi_MSA_both, mag_vocab, msa_vocab)

en_vocab.close()
msa_vocab.close()
lav_vocab.close()
mag_vocab.close()

In [0]:
VOCAB_SIZE = 32128
spm.SentencePieceTrainer.train('--input=data/vocab/eng_vocab.txt,data/vocab/msa_vocab.txt,data/vocab/lav_vocab.txt,data/vocab/mag_vocab.txt --model_prefix=data/model/spm --vocab_size=' + str(VOCAB_SIZE))

In [104]:
sp = spm.SentencePieceProcessor()
sp.load('data/model/spm.model')

True

In [105]:
print(sp.encode_as_pieces('This is a test'))
print(sp.encode_as_ids('This is a test'))

['▁This', '▁', 'is', '▁', 'a', '▁', 't', 'est']
[552, 3, 100, 3, 76, 3, 52, 3071]


In [106]:
print(sp.encode_as_pieces('هناك شوكة ناقصة'))
print(sp.encode_as_ids('هناك شوكة ناقصة'))

['▁هناك', '▁', 'شوكة', '▁', 'ناقصة']
[396, 3, 10425, 3, 10999]


In [107]:
print(sp.encode_as_pieces('This is a test'))
print(sp.encode_as_ids('This is a test'))

['▁This', '▁', 'is', '▁', 'a', '▁', 't', 'est']
[552, 3, 100, 3, 76, 3, 52, 3071]


In [108]:
sp.tokenize("This is a test")

[552, 3, 100, 3, 76, 3, 52, 3071]

Pytorch Dataset

In [0]:
#https://pytorch.org/text/_modules/torchtext/data/field.html
class CustomField(Field):
  
  def __init__(self, tokenize=None, init_token=None, eos_token=None, lower=False):
        Field.__init__(self)
        self.tokenize = tokenize

  def preprocess(self, x):
    #print(x)
    x = sp.tokenize((x.rstrip('\n')))
    return x
  


In [0]:
SRC = CustomField(tokenize = sp,
            init_token = '<sos>',
            eos_token = '<eos>',
            lower = False)

TRG = CustomField(tokenize = sp,
            init_token = '<sos>',
            eos_token = '<eos>',
            lower = False)

SRC.use_vocab = True
TRG.use_vocab = True

In [111]:
print(SRC.tokenize.tokenize("this is a test"))
callable(SRC.tokenize.tokenize("this is a test"))

[3, 245, 3, 100, 3, 76, 3, 52, 3071]


False

In [0]:
eng_msa_train_dataset = TranslationDataset(path='data/train/train_eng_msa.', exts=('eng', 'msa'), fields=(SRC, TRG))
lav_msa_train_dataset = TranslationDataset(path='data/train/train_lav_msa.', exts=('lav', 'msa'), fields=(SRC, TRG))
mag_msa_train_dataset = TranslationDataset(path='data/train/train_mag_msa.', exts=('mag', 'msa'), fields=(SRC, TRG))

In [0]:
SRC.build_vocab(eng_msa_train_dataset, lav_msa_train_dataset, mag_msa_train_dataset)
TRG.build_vocab(eng_msa_train_dataset, lav_msa_train_dataset, mag_msa_train_dataset)

In [0]:
train_data, valid_data, test_data = eng_msa_train_dataset.splits(path= 'data/', train='train/train_eng_msa', validation='val/val_eng_msa', test='test/test_eng_msa', exts=('.eng', '.msa'),
                                                    fields = (SRC, TRG))

In [115]:
print(train_data[10].src)

[129, 248, 3, 1382, 20]


In [0]:
BATCH_SIZE =32

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

train_iterator, valid_iterator, test_iterator = BucketIterator.splits(
    (train_data, valid_data, test_data),
    batch_size = BATCH_SIZE,
    device = device,
    shuffle=True)

In [0]:
pad_idx = TRG.vocab.stoi['<pad>']
eos_idx = TRG.vocab.stoi['<eos>']
sos_idx = TRG.vocab.stoi['<sos>']

criterion = nn.CrossEntropyLoss(ignore_index=pad_idx)

In [130]:
print(pad_idx)
print(eos_idx)
print(sos_idx)

1
0
0


In [0]:
def train(model: nn.Module,
          iterator: BucketIterator,
          optimizer: optim.Optimizer,
          criterion: nn.Module,
          clip: float):

    model.train()

    epoch_loss = 0

    for _, batch in enumerate(iterator):

        src = batch.src
        trg = batch.trg

        optimizer.zero_grad()

        output = model(src, trg)

        output = output[1:].view(-1, output.shape[-1])
        trg = trg[1:].view(-1)

        loss = criterion(output, trg)

        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)

        optimizer.step()

        epoch_loss += loss.item()

    return epoch_loss / len(iterator)

In [0]:
def evaluate(model: nn.Module,
             iterator: BucketIterator,
             criterion: nn.Module):

    model.eval()

    epoch_loss = 0

    with torch.no_grad():

        for _, batch in enumerate(iterator):

            src = batch.src
            trg = batch.trg

            output = model(src, trg, 0) #turn off teacher forcing?

            output = output[1:].view(-1, output.shape[-1])
            trg = trg[1:].view(-1)

            loss = criterion(output, trg)

            epoch_loss += loss.item()

    return epoch_loss / len(iterator)

##XLM Model

In [120]:
xlmr = torch.hub.load('pytorch/fairseq', 'xlmr.base')
xlmr.train();

Using cache found in /root/.cache/torch/hub/pytorch_fairseq_master


##T5 Model

In [121]:
tokenizer = T5Tokenizer.from_pretrained('t5-small')
model = T5Model.from_pretrained('t5-small')
input_ids = tokenizer.encode("Hello, my dog is cute", return_tensors="pt")  # Batch size 1
print(input_ids)
outputs = model(input_ids=input_ids, decoder_input_ids=input_ids)
last_hidden_states = outputs[0]  # The last hidden-state is the first element of the output tuple
print(last_hidden_states)

tensor([[8774,    6,   82, 1782,   19, 5295]])
tensor([[[ 9.2098e-02,  1.1048e-01,  2.6714e-02,  ...,  1.2918e-02,
           6.1260e-05,  9.5353e-02],
         [ 8.7042e-02,  8.3914e-02,  6.9337e-02,  ..., -3.9229e-02,
           3.3525e-04,  1.4291e-01],
         [ 9.6290e-02, -4.8916e-03,  5.5687e-02,  ..., -1.0703e-01,
           6.4940e-04, -2.1393e-01],
         [-3.0119e-03,  1.1048e-01,  3.0695e-03,  ..., -5.1768e-02,
           3.5166e-04,  1.5510e-01],
         [-6.3620e-02,  5.4474e-02, -1.8415e-02,  ..., -8.4559e-02,
           6.1696e-04,  5.8805e-02],
         [-6.0232e-02,  1.3885e-01,  7.9864e-03,  ..., -4.9981e-02,
           4.3370e-04,  4.4866e-02]]], grad_fn=<MulBackward0>)


In [122]:
tokenizer = sp
model = T5Model.from_pretrained('t5-small')
input_ids = tokenizer.encode_as_ids("Hello, my dog is cute")  # Batch size 1
input_ids = torch.LongTensor([input_ids])
print(input_ids)
outputs = model(input_ids=input_ids, decoder_input_ids=input_ids)
last_hidden_states = outputs[0]  # The last hidden-state is the first element of the output tuple
print(last_hidden_states)

tensor([[  121,   368,   416,     0,     3,   303,     3,  3747,     3,   100,
             3, 26278]])
tensor([[[-8.4151e-02,  1.3744e-01,  3.4957e-02,  ..., -6.9336e-02,
           3.8151e-04, -8.3164e-02],
         [ 1.9653e-02,  2.9976e-01,  8.3203e-02,  ...,  6.1443e-03,
           2.8821e-04, -5.1392e-02],
         [ 4.4363e-02,  2.7316e-01,  5.3030e-02,  ...,  3.6395e-03,
           5.4526e-04, -3.5499e-02],
         ...,
         [ 1.0066e-01,  3.6209e-01, -7.6802e-02,  ..., -2.0994e-02,
           3.2503e-04, -1.8582e-01],
         [ 6.1817e-02,  3.0828e-01, -5.9316e-02,  ..., -1.8453e-03,
           3.1223e-04, -1.0168e-01],
         [ 5.8691e-02,  3.3793e-01, -4.1354e-02,  ...,  4.3149e-03,
           2.0226e-04, -1.5124e-01]]], grad_fn=<MulBackward0>)


In [0]:
config = T5Config(vocab_size=VOCAB_SIZE)
model = T5Model(config)
optimizer = torch.optim.AdamW(model.parameters(), lr=0.01)

In [124]:
config

T5Config {
  "d_ff": 2048,
  "d_kv": 64,
  "d_model": 512,
  "dropout_rate": 0.1,
  "eos_token_id": 1,
  "initializer_factor": 1.0,
  "is_encoder_decoder": true,
  "layer_norm_epsilon": 1e-06,
  "model_type": "t5",
  "n_positions": 512,
  "num_heads": 8,
  "num_layers": 6,
  "pad_token_id": 0,
  "relative_attention_num_buckets": 32,
  "vocab_size": 32128
}

## Model Training Code

In [0]:
model = T5Model.from_pretrained('t5-small')
#model = xlmr
optimizer = optim.AdamW(model.parameters(), lr=0.0001)

In [125]:
N_EPOCHS = 10
CLIP = 1

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):

    start_time = time.time()

    train_loss = train(model, train_iterator, optimizer, criterion, CLIP)
    valid_loss = evaluate(model, valid_iterator, criterion)

    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)

    print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):7.3f}')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. PPL: {math.exp(valid_loss):7.3f}')

test_loss = evaluate(model, test_iterator, criterion)

print(f'| Test Loss: {test_loss:.3f} | Test PPL: {math.exp(test_loss):7.3f} |')

RuntimeError: ignored